In [10]:
%pylab inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn import model_selection, datasets, metrics
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_extraction import DictVectorizer as DV

from sklearn.tree import export_graphviz

from sklearn.feature_selection import RFE

from sklearn.ensemble import RandomForestClassifier

Populating the interactive namespace from numpy and matplotlib


In [11]:
%matplotlib inline

In [12]:
data = pd.read_csv('./kaggle/orange_small_churn_train_data.csv', sep = ',', header = 0, index_col=0)
test_data = pd.read_csv('./kaggle/orange_small_churn_test_data.csv', sep = ',', header = 0, index_col=0)

## Подготовка данных

In [13]:
# т.к. в последней строке тренировочных данных ответа нету, то удаляем её
train_data = data.iloc[:-1,:-1]
train_labels = data.iloc[:-1,-1:]

print train_data.shape
print train_labels.shape
print test_data.shape

(18298, 230)
(18298, 1)
(10000, 230)


In [14]:
# отделяем числовые и категориальные признаки и удаляем полностью пустые признаки
data_numb = train_data.iloc[:,0:190].dropna(axis=1, how='all')
data_categ = train_data.iloc[:,190:].dropna(axis=1, how='all')

In [15]:
columns_name = data_numb.columns.to_list()

In [16]:
# заменим NONE на среднее значение колонки
# Посчитаем средние по колонкам
numeric_means = data_numb.mean(axis=0, skipna=True)

# Заполним пропущенные численные значения средними
data_numb = data_numb.fillna(numeric_means, axis=0)

In [8]:
# попробуем обучить только вещественные признаки чтобы понять какие признакие самые важные
select = RFE(RandomForestClassifier(n_estimators = 100, random_state=0, learning_rate=0.1, max_depth=1),
            n_features_to_select=10)

NameError: name 'GradientBoostingClassifier' is not defined

In [17]:
support_ = array([False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False])

In [346]:
%%time
select.fit(data_numb, train_labels)

CPU times: user 4min 5s, sys: 120 ms, total: 4min 5s
Wall time: 4min 5s


RFE(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
  n_features_to_select=10, step=1, verbose=0)

In [18]:
support_

array([False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [19]:
columns_top10=[columns_name[ind] for ind, col in enumerate(support_) if col]

In [20]:
data_numb = data_numb[columns_top10]

In [21]:
# ВЕЩЕСТВЕННЫЕ признаки
# NOTE: сделаем скелинг численных признаков с D=1 (подробнее см. неделю 1)
# NOTE2: такая запись по созданию нового DataFrame нужна из-за тоог что при fit_transform слетают индексы и потом
# при контатенации у нас получается каша.
scaler = StandardScaler()
data_numb=pd.DataFrame(scaler.fit_transform(data_numb.values), index=data_numb.index, columns=columns_top10)

# меняем пустые на 0 (0 это среднее так как признаки масштабированы)
# data_numb.fillna(0, inplace=True)


# КАТЕГОРИАЛЬНЫЕ признаки
# заменим пустые значения на NA (будет как доп.признак)
data_categ = data_categ.fillna('NA').applymap(lambda s: str(s))

# удалим те колонки где больше 20 категорий и меньше 2 (эти колонки не информативны)
name_del = [name for name, var in data_categ.iteritems() if var.value_counts(dropna=True).shape[0] > 20 or var.value_counts(dropna=True).shape[0] < 2]
data_categ = data_categ.drop(labels=name_del, axis=1)

In [22]:
# ВАРИАНТ 1 кодируем категориальные признаки
# data_dummies = pd.get_dummies(data_categ)


# ВАРИАНТ 2 one-hote-encoder
encoder = DV(sparse = False)
data_dummies = encoder.fit_transform(data_categ.T.to_dict().values())

In [23]:
data_dummies.shape

(18298, 101)

In [24]:
# при ВАРИАНТЕ 1
# объединяем числовые признаки и закодированные категориальные
# train_data = pd.concat([data_numb, data_dummies], axis=1)
# train_data.fillna(0, inplace=True)


# при ВАРИАНТЕ 2
train_data = np.hstack((data_numb.values, data_dummies))
# train_data.fillna(0, inplace=True)

In [25]:
print data_numb.shape
print data_categ.shape

print train_data.shape
print train_labels.shape

(18298, 10)
(18298, 20)
(18298, 111)
(18298, 1)


## Случайный лес

In [32]:
from sklearn.ensemble import RandomForestClassifier

# max_depth - максимальная глубина
# learning_rate - насколько сильно каждое дерево будет пытаться исправить ошибки предыдущих деревьев.
parameters_grid = {
    'n_estimators' : range(70, 130, 5),
    'max_depth': range(1, 8, 1),
    'max_features': ['sqrt', 'log2'],
}

# Будем использовать метод стратификации который делит соотношение классов в обучающей выборке на равное количество
skf = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
classifier = RandomForestClassifier(random_state=0, class_weight = 'balanced', n_jobs=2)
grid_rfc = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'roc_auc', cv = skf)

In [33]:
%%time
grid_rfc.fit(train_data, train_labels)

CPU times: user 7min 55s, sys: 22.4 s, total: 8min 17s
Wall time: 15min 22s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125], 'max_features': ['sqrt', 'log2'], 'max_depth': [1, 2, 3, 4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [30]:
grid_rfc.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=105, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [31]:
print grid_rfc.best_score_
print grid_rfc.best_params_

0.7112294421529325
{'n_estimators': 105, 'max_depth': 6}


## Градиентный бустинг деревьев

In [381]:
from sklearn.ensemble import GradientBoostingClassifier

# max_depth - максимальная глубина
# learning_rate - насколько сильно каждое дерево будет пытаться исправить ошибки предыдущих деревьев.
parameters_grid = {
    'n_estimators' : range(80, 120, 5),
    'learning_rate' : [0.05, 0.1, 0.07, 0.08, 0.09],
    'max_depth': range(1, 5, 1)
}

# Будем использовать метод стратификации который делит соотношение классов в обучающей выборке на равное количество
skf = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
classifier = GradientBoostingClassifier(random_state=0)
grid_rfc = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'roc_auc', cv = skf)

In [29]:
%%time
grid_rfc.fit(train_data, train_labels)

CPU times: user 1h 9min 17s, sys: 8.38 s, total: 1h 9min 25s
Wall time: 1h 3min 12s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [90, 95, 100, 105, 110], 'learning_rate': [0.05, 0.1, 0.07, 0.08, 0.09], 'max_depth': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [30]:
grid_rfc.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=110,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [31]:
print grid_rfc.best_score_
print grid_rfc.best_params_

0.7251141392953343
{'n_estimators': 110, 'learning_rate': 0.1, 'max_depth': 1}


-----

In [355]:
# строим модель с оптимальными параметрами которые удалось подобрать
clf = GradientBoostingClassifier(n_estimators = 110, random_state=0, learning_rate=0.1, max_depth=1)
clf.fit(train_data, train_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=110,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [356]:
# проверяем метрики на тренировочном наборе
actual_labels = clf.predict(train_data)
# actual_labels_proba = clf.predict_proba(train_data)

In [378]:
print "AUC_ROC =", metrics.roc_auc_score(train_labels, actual_labels)
print "accuracy =", clf.score(train_data, train_labels)
print "precision =", metrics.precision_score(train_labels, actual_labels)
print "recall =", metrics.recall_score(train_labels, actual_labels)
print "f1 =", metrics.f1_score(train_labels, actual_labels)

AUC_ROC = 0.5021491001564492
accuracy = 0.9250191277735271
precision = 0.8571428571428571
recall = 0.004357298474945534
f1 = 0.008670520231213874


In [ ]:
# AUC_ROC = 0.5021491001564492
# accuracy = 0.9250191277735271
# precision = 0.8571428571428571
# recall = 0.004357298474945534

In [358]:
# важность признаков
clf.feature_importances_

array([0.        , 0.05567181, 0.05632596, 0.05569146, 0.0143665 ,
       0.0820897 , 0.30228797, 0.00563952, 0.03133637, 0.11886813,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0541655 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00984846, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00289404,
       0.01003713, 0.00858656, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.1628948 , 0.        ,
       0.02590311, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

## Тестовый набор

In [365]:
# Предобрабатываем тестовый набор
print test_data.shape

# отделяем числовые и категориальные признаки и удаляем полностью пустые признаки
# data_numb_test = test_data.iloc[:,0:190].dropna(axis=1, how='all')
data_numb_test = test_data[columns_top10]
data_categ_test = test_data.iloc[:,190:]

# data_dummies = pd.get_dummies(data_categ)


(10000, 230)


In [366]:
# ВЕЩЕСТВЕННЫЕ признаки
numeric_means_test = data_numb_test.mean(axis=0, skipna=True)
# Заполним пропущенные численные значения средними
data_numb_test = data_numb_test.fillna(numeric_means_test, axis=0)

data_numb_test =pd.DataFrame(scaler.transform(data_numb_test.values), index=data_numb_test.index, columns=columns_top10)

# data_numb_test.fillna(0, inplace=True)

# КАТЕГОРИАЛЬНЫЕ ПРИЗНАКИ
# берем только колонки используемые в обучении
data_categ_test = data_categ_test[data_categ.columns]
data_categ_test = data_categ_test.fillna('NA').applymap(lambda s: str(s))
# удалим те колонки где больше 20 категорий и меньше 2 (эти колонки не информативны)
# name_del = [name for name, var in data_categ_test.iteritems() if var.value_counts(dropna=True).shape[0] > 20 or var.value_counts(dropna=True).shape[0] < 2]
# data_categ_test = data_categ_test.drop(labels=name_del, axis=1)

In [367]:
print data_numb_test.shape
print data_categ_test.shape

(10000, 10)
(10000, 20)


In [368]:
# кодируем категориальные признаки
# data_dummies_test = pd.get_dummies(data_categ_test)

data_dummies_test = encoder.transform(data_categ_test.T.to_dict().values())
# data_dummies_test = np.where(data_dummies_test == np.nan, data_dummies_test, 0)

In [370]:
# объединяем числовые признаки и закодированные категориальные
test_data = np.hstack((data_numb_test.values, data_dummies_test))

In [371]:
print test_data.shape

(10000, 111)


In [372]:
data_dummies_test[:1]

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0.]])

In [373]:
test_labels = clf.predict(test_data)
test_labels_proba = clf.predict_proba(test_data)

## Сохраняем результат для Kaggle

In [374]:
df = pd.DataFrame(test_labels_proba[:,1], columns=['result'])  
df.index.name = 'ID'
# df['index'] = df.index
# df.astype({"ID": int, "result": float})

In [375]:
# np.savetxt('output.csv', df, delimiter=',', fmt='%f', header='ID, result')
df.to_csv('output.csv', index=True)